# Patent Analysis

In [1]:
%%capture
# !pip install -r requirements.txt


## Importing Libraries

In [3]:
%%capture
import sys
import importlib
sys.path.append('src')


import preprocess_data
import qaagent
import koragent

importlib.reload(preprocess_data)
importlib.reload(qaagent)
importlib.reload(koragent)


In [4]:

import nltk
nltk.download('all', quiet=True)

True

## Downloading patents and preprocessing

In [5]:
year = 2023
month = 1
day = 5

In [6]:
saved_patent_names = preprocess_data.parse_and_save_patents(year, month, day, False)

## Loading data and querying sample file


In [16]:
from kor.nodes import Object, Text

schema = Object(
    id="physical_measurements",
    description="Details about a physical measurements in text",
    attributes=[
        Text(
            id="substance",
            description="Substance that was measured",
            examples=[
                ("The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", "BaCO3"),
                ("Mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm", "Mesoporous silicon dioxide nanoparticles")
            ],
        ),
        Text(
            id="value",
            description="Value of the measurement",
            examples=[
                ("The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", "between 20 and 40"),
                ("Mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm", "between 5 and 15")
            ],
        ),
        Text(
            id="unit",
            description="Unit of the measurement",
            examples=[
                ("The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", "nm"),
                ("Mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm", "nm")
            ],
        ),
    ],
    examples=[
        (
            "The material for the support can be mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm",
            [
                {"substance": "mesoporous silicon dioxide nanoparticles", "measurement_value": "between 5 and 15", "unit": "nm"}
            ],
        ),
        (
            "The resulting BaCO3 had a crystallite size of between about 20 and 40 nm",
            [
                {"substance": "BaCO3", "measurement_value": "between 20 and 40", "unit": "nm"}
            ],
        )
    ],
    many=True,
)

schema = Object(
    id="physical_measurements",
    description="Details about physical measurements in text",
    attributes=[
        Text(
            id="substance",
            description="Substance that was measured",
            examples=[
                ("The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", "BaCO3"),
                ("Mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm", "Mesoporous silicon dioxide nanoparticles")
            ],
        ),
        Text(
            id="measurement_type",
            description="The type of measurement being conducted (e.g., diameter, size, etc.)",
            examples=[
                ("The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", "crystallite size"),
                ("Mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm", "pore diameter"),
            ],
        ),
        Text(
            id="value",
            description="Value of the measurement",
            examples=[
                ("The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", "between 20 and 40"),
                ("Mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm", "between 5 and 15")
            ],
        ),
        Text(
            id="unit",
            description="Unit of the measurement",
            examples=[
                ("The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", "nm"),
                ("Mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm", "nm")
            ],
        ),
    ],
    examples=[
        (
            "The material for the support can be mesoporous silicon dioxide nanoparticles with an average pore diameter between 5 nm  and 15 nm",
            [
                {"substance": "mesoporous silicon dioxide nanoparticles", "measurement_type": "pore diameter", "value": "between 5 and 15", "unit": "nm"}
            ],
        ),
        (
            "The resulting BaCO3 had a crystallite size of between about 20 and 40 nm",
            [
                {"substance": "BaCO3", "measurement_type": "crystallite size", "value": "between 20 and 40", "unit": "nm"}
            ],
        )
    ],
    many=True,
)


In [13]:
import json
import os

# Creating an empty dictionary to store the JSON data
json_files = {}

# Specifying the path to the JSON files
folder_path = os.path.join(
        os.getcwd(),
        "output"
    )


# Iterating through every file in the directory
for file_name in os.listdir(folder_path):
    # Checking if the current file is a JSON file
    if file_name.endswith('.json'):
        # Defining the path to the file
        file_path = os.path.join(folder_path, file_name)

        # Opening the JSON file
        with open(file_path, 'r') as file:
            # Loading the JSON data from the file
            data = json.load(file)

            # Adding the JSON data to the dictionary
            # The key is the file name split at the first "-" and taking the first part
            key_name = file_name.split('-')[0]
            json_files[key_name] = data


# Printing the dictionary to check if the data was loaded correctly
poc = []
for key in json_files:
    poc.append(key)



JSONDecodeError: Expecting value: line 8 column 9 (char 1064)

In [14]:
indices = []
for poc_item in poc:
    for i, patent_path in enumerate(saved_patent_names):
        if poc_item in patent_path:
            indices.append(i)

In [22]:
importlib.reload(koragent)
raw, output = await koragent.call_extraction_to_json(schema, year, month, day, saved_patent_names, 96, True, "gpt-3.5-turbo")

Starting the extraction process...
Loading documents from: c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105\US20230001042A1-20230105.XML.txt
Running extraction chain...


Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} <CIMultiDictProxy('Date': 'Tue, 01 Aug 2023 01:32:55 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7efa4ce64d6ba1e0-YYZ', 'alt-svc': 'h3=":443"; ma=86400')>.
c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\.conda\Lib\site-packages\tenacity\__init__.py:338: RuntimeWarning: coroutine 'AsyncRunManager.on_retry' was never awaited
  self.before_sleep(retry_state)


Total Tokens: 32615
Prompt Tokens: 27962
Completion Tokens: 4653
Successful Requests: 32
Total Cost (USD): $0.051249
{'uid': '0', 'source_uid': '0', 'data': {}, 'raw': 'substance|measurement_type|value|unit\n|||\nmeasurement_type|value|unit\nsuture thread|curled||\nresin materials|improving operability||\nresin materials|slidability||\nresin materials|slide-down performance||\nMPC|enhance easiness||\nMPC|retainability||\nsuture thread|coated with|MPC||\npackaging|||', 'validated_data': {}, 'errors': [ParseError(ParserError('Error tokenizing data. C error: Expected 4 fields in line 10, saw 5\n'))]}
Writing the output to a file...


TypeError: Object of type ParseError is not JSON serializable

In [17]:
importlib.reload(koragent)

results= {}
for idx in indices:
    raw, output = await koragent.call_extraction_to_json(schema, year, month, day, saved_patent_names, idx, True, "gpt-3.5-turbo")
    results[idx] = [raw, output]

Starting the extraction process...
Loading documents from: c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105\US20230001042A1-20230105.XML.txt
Running extraction chain...
{'uid': '0', 'source_uid': '0', 'data': {'physical_measurements': [{'substance': 'medical suture thread', 'measurement_type': '', 'value': '', 'unit': ''}, {'substance': 'outer surface coating', 'measurement_type': '', 'value': '', 'unit': ''}, {'substance': 'resin materials', 'measurement_type': '', 'value': '', 'unit': ''}, {'substance': 'operability improvement', 'measurement_type': '', 'value': '', 'unit': ''}, {'substance': 'slidability improvement', 'measurement_type': '', 'value': '', 'unit': ''}, {'substance': 'slide-down performance improvement', 'measurement_type': '', 'value': '', 'unit': ''}, {'substance': '2-methacryloyloxyethyl phosphorylcholine (MPC) coating', 'measurement_type': '', 'value': '', 'unit': ''}]}, 'raw': 'substance|measurement_type|value|unit\r\nmedical suture thread|||\

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001D50DA71850>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001D5CA6FAA50>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001D50DA41110>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001D5C9EACB10>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001D5CA1922D0>


CancelledError: 